In [6]:
dataset_name = "W14_comb"
df_list = ['BES_Panel']#,"BESnumeric"]

In [7]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re

from IPython.display import display, display_html 

import Jupyter_module_loader
from utility import *

# you should clone this git to a subdirectory called 'BES_analysis_code' (in some directory - I call it BES_analysis - doesn't matter though)
%matplotlib inline
encoding = "ISO-8859-1"

(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()


global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

for df in df_list:
    if df=="BES_Panel":
        globals()[df]  = pd.read_msgpack(data_subfolder + dataset_filename.replace('.dta','.msgpack'))

    else:
        globals()[df]  = pd.read_msgpack(data_subfolder + df + '.msgpack' )
        globals()[df].replace(-1,np.nan,inplace=True)

(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)

memory_use(locs = locals().items())

var_type (5692, 13)
                     BES_Panel: 506.4MiB
                      var_type:   4.3MiB
               BES_census_data:   2.0MiB
             new_old_col_names: 384.1KiB
             old_new_col_names: 192.1KiB
                cat_dictionary: 192.1KiB
             BES_file_manifest:  42.4KiB
                      manifest:   1.7KiB
                           _i3:   1.3KiB
                           _i7:   1.3KiB


In [8]:
max_wave, num_to_wave, num_to_weight, weights = get_weights(dataset_name,BES_Panel)

In [22]:
BES_census_data = pd.read_stata( BES_small_data_files + "BES-2017-General-Election-results-file-v1.0.dta" )
print("BES_census_data", BES_census_data.shape )

BES_census_data (632, 308)


In [11]:
corrs = BES_census_data.corrwith(BES_census_data["leaveHanretty"]).sort_values()

In [12]:
corrs[search(BES_census_data, 'Age')]
# p_surridge note - hanretty const. Leave est. derived using MRP which uses age data

c11Age0to4     -0.021106
c11Age5to7      0.144198
c11Age8to9      0.210472
c11Age10to14    0.406411
c11Age15        0.508010
c11Age16to17    0.527042
c11Age18to19   -0.217939
c11Age20to24   -0.379248
c11Age25to29   -0.493926
c11Age30to44   -0.468753
c11Age45to59    0.348489
c11Age60to64    0.440701
c11Age65to74    0.446599
c11Age75to84    0.388374
c11Age85to89    0.314405
c11Age90plus    0.223094
dtype: float64

In [53]:
def make_corr_summary(input_df, name, corr_type = "spearman", sample_size_text = "N", correlation_text = "r",
                      abs_correlation_text = "abs_r", p_value_text = "p",
                      min_p_value = 0.01, min_variance = 0.0, min_sample_size = 500):

    df1 = input_df.copy()
    focal_var = df1[name]
    focal_mask = focal_var.notnull()


    pattern_list = [x for x in df1.columns if re.search(pattern,x)]

    variances = df1[focal_mask].var()
    low_var_list = list(variances[variances<min_variance].index)
    sample_sizes = df1[focal_mask].notnull().sum()
    low_sample_size_list = list(sample_sizes[sample_sizes<min_sample_size].index)

    drop_list = pattern_list+low_var_list+low_sample_size_list
    df1.drop(drop_list,axis=1,inplace=True)

    if corr_type == "pearson":
        df = df1.apply(lambda x: corr_simple_pearsonr(x,focal_var)).apply(pd.Series).T
    elif corr_type == "spearman":
        df = df1.apply(lambda x: corr_simple_spearmanr(x,focal_var)).apply(pd.Series).T

    df.columns = [correlation_text,p_value_text,sample_size_text]
#     return(sample_size_text)
#     return(df)    
    df[sample_size_text] = df[sample_size_text].astype('int')
    df[abs_correlation_text] = df[correlation_text].abs()

    zero_var_other_way_around_list = list(df[df[correlation_text].isnull()].index)
    df.dropna(inplace=True)

    insignificant_list = df[df[p_value_text]>min_p_value].index
    df.drop(insignificant_list,inplace=True)

    df.sort_values(by=abs_correlation_text,ascending=False,inplace=True)


    stub_dict = {}
    drop_list = []
    # drop repeated references to same variable in different waves???
    # so, what about different categories??? eg. blahWX_subcat
    # how about, just replace wave match as "X"
    # create a dictionary keyed on the top corr variable with all the drops inside
    for ind in df.index:
        waveless = remove_wave(ind)
        if waveless in stub_dict.keys():
            drop_list.append(ind)
            stub_dict[waveless].append(ind)
        else:
            stub_dict[waveless] = [ind]
    df.drop(drop_list,inplace=True)
    return df, corr_type

In [56]:
def display_corr(df, name, corr_type, top_num = 20, round_places = 2,
                 correlation_text = "r", p_value_text = "p", sample_size_text = "N"):
    df.index = [x[0:60] for x in df.index]
    df[correlation_text] = df[correlation_text].round(round_places)
    
    df1 = df.sort_values(by=correlation_text, ascending=False)[0:top_num][[correlation_text,p_value_text,sample_size_text]]
    df2 = df.sort_values(by=correlation_text)[0:top_num][[correlation_text,p_value_text,sample_size_text]]
    
    df1[p_value_text]     = df1[p_value_text].apply(lambda x: "{0:0.2f}".format(x))
    df2[p_value_text]     = df2[p_value_text].apply(lambda x: "{0:0.2f}".format(x))

    df1_caption = "Top "+str(top_num)+ " positive "+"("+corr_type+")"+" correlations for "+name
    df2_caption = "Top "+str(top_num)+ " negative "+"("+corr_type+")"+" correlations for "+name

    df1_styler = df1.style.set_table_attributes("style='display:inline'").set_caption(df1_caption)
    df2_styler = df2.style.set_table_attributes("style='display:inline'").set_caption(df2_caption)

    display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)


In [61]:


pattern = "remainHanretty|leaveHanretty"
name = "leaveHanretty"
corr_type = "spearman"
df,corr_type = make_corr_summary(BES_census_data, name, corr_type)
display_corr(df, name, corr_type)


C:\Users\Marios\Anaconda3\envs\BES_analysis\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


,r,p,N
UKIP15,0.88,0.00,614
UKIP1015,0.84,0.00,614
UKIPVote15,0.78,0.00,614
c11HouseholdCohabitDependents,0.72,0.00,632
c11HealthFair,0.68,0.00,632
c11PassportNone,0.68,0.00,573
c11IndustryManufacturing,0.65,0.00,632
c11NSSECSemiRoutine,0.65,0.00,632
c11HouseholdCohabitNodependents,0.63,0.00,632
c11NSSECLowerSupervisor,0.62,0.00,632
